# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [22]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [23]:
df = pd.read_csv("Output/city_weather.csv")
df

,City,Country,Lat,Lang,Max Temp,Humidity,Cloudiness,Wind Speed,Date
0,albany,au,42.6001,-73.9662,84.54,75,46,3.06,1627247018
1,oranjestad,aw,12.5240,-70.0270,85.95,70,20,10.36,1627247633
2,ballarat,au,-37.5662,143.8496,48.18,94,99,8.99,1627247634
3,jamestown,sh,42.0970,-79.2353,84.09,57,1,11.50,1627247016
4,rikitea,pf,-23.1203,-134.9692,70.88,67,25,5.03,1627247027
...,...,...,...,...,...,...,...,...,...
537,alyangula,au,-13.8483,136.4192,69.94,73,93,11.07,1627247851
538,tiarei,pf,-17.5333,-149.3333,82.42,68,1,8.81,1627247852
539,sardarshahr,in,28.4333,74.4833,92.70,53,58,11.32,1627247852
540,petropavlovsk-kamchatskiy,ru,53.0452,158.6483,63.19,88,0,6.71,1627247852


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [36]:
# Configure gmaps with API key
gmaps.__version__

gmaps.configure(api_key=g_key)

In [38]:
# Store 'Lat' and 'Lng' into  locations 
locations = df[["Lat", "Lang"]].astype(float)
locations

lats = locations["Lat"]
lngs = locations["Lang"]
loc_symbols = zip(lats,lngs)

# # loc_symbols = [(lat, lng) for lat, lng in locations]
# loc_symbols = []
# for index,row in locations.iterrows():
#     print(index)
# #     loc_symbols.append((locations.iloc[index,0], locations.iloc[index,1]))
loc_symbols

# Humidity as weight
humidity = df["Humidity"].astype(float)

# Create a Humidity Heatmap layer
morocco_coordinates = (31.43939795909406, -6.103582692418146)
fig = gmaps.figure(center = morocco_coordinates, zoom_level=2)

heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 3)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

In [21]:
# Create bank symbol layer
hum_info_layer = gmaps.symbol_layer(
    locations,
#     fill_color='rgba(0, 150, 0, 0.4)',
#     stroke_color='rgba(0, 0, 150, 0.4)', scale=2,
    info_box_content=[f"Locations: {loc_symbol}" for loc_symbol in loc_symbols]
)

fig = gmaps.figure()
fig.add_layer(hum_info_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [43]:
# Create DataFrame for the ideal weather conditions
# Max temperature lower than 80 degrees but higher than 70.
# Wind speed less than 10 mph.
# Zero cloudiness.

ideal_weather = df.loc[(df["Max Temp"]>70) & (df["Max Temp"]<80) & (df["Wind Speed"]<10) & (df["Cloudiness"]==0)]
ideal_weather

#   Drop any rows that don't contain all three conditions. You want to be sure the weather is ideal.

,City,Country,Lat,Lang,Max Temp,Humidity,Cloudiness,Wind Speed,Date
30,gardez,af,33.5974,69.2259,72.75,49,0,3.22,1627247643
40,paveh,ir,35.0434,46.3565,77.07,27,0,2.93,1627247646
48,mangaratiba,br,-22.9597,-44.0406,71.02,68,0,3.40,1627247648
107,urumqi,cn,43.8010,87.6005,77.95,28,0,4.47,1627247203
110,ponta do sol,cv,32.6667,-17.1000,77.00,83,0,4.76,1627247071
117,tiznit,ma,29.5833,-9.5000,79.52,35,0,3.22,1627247669
122,monte alegre de minas,br,-18.8706,-48.8808,72.37,29,0,5.57,1627247671
165,guapiacu,br,-20.7950,-49.2203,73.22,25,0,6.06,1627247687
274,luzhou,cn,28.8724,105.4405,75.33,93,0,1.61,1627247485
338,luziania,br,-16.2525,-47.9503,76.69,34,0,5.53,1627247762


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [ ]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [ ]:
# Add marker layer ontop of heat map


# Display figure
